In [1]:
import json
import pandas as pd
df = pd.read_json(r'Musical_Instruments_5.json',lines = True)
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014"
...,...,...,...,...,...,...,...,...,...
10256,A14B2YH83ZXMPP,B00JBIVXGC,Lonnie M. Adams,"[0, 0]","Great, just as expected. Thank to all.",5,Five Stars,1405814400,"07 20, 2014"
10257,A1RPTVW5VEOSI,B00JBIVXGC,Michael J. Edelman,"[0, 0]",I've been thinking about trying the Nanoweb st...,5,"Long life, and for some players, a good econom...",1404259200,"07 2, 2014"
10258,AWCJ12KBO5VII,B00JBIVXGC,Michael L. Knapp,"[0, 0]",I have tried coated strings in the past ( incl...,4,Good for coated.,1405987200,"07 22, 2014"
10259,A2Z7S8B5U4PAKJ,B00JBIVXGC,"Rick Langdon ""Scriptor""","[0, 0]","Well, MADE by Elixir and DEVELOPED with Taylor...",4,Taylor Made,1404172800,"07 1, 2014"


In [14]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10261 entries, 0 to 10260
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      10261 non-null  object 
 1   asin            10261 non-null  object 
 2   reviewerName    10234 non-null  object 
 3   helpful         10261 non-null  object 
 4   reviewText      10261 non-null  object 
 5   overall         10261 non-null  int64  
 6   summary         10261 non-null  object 
 7   unixReviewTime  10261 non-null  int64  
 8   reviewTime      10261 non-null  object 
 9   ratings         10261 non-null  object 
 10  scores          10261 non-null  object 
 11  compound        10261 non-null  float64
 12  sentiment_type  10261 non-null  object 
dtypes: float64(1), int64(2), object(10)
memory usage: 1.0+ MB


Considering entire dataset for Sentiment Analysis using Vader Sentiment Analyzer

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

def preprocessing(col):
    
    # lowercasing
    df[col] = df[col].str.lower()

    # Remove punctuation
    df[col] = df[col].str.replace('[^\w\s]','', regex=True)
    
    #Lemmatization
    df[col] = df[col].apply(lambda x : ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
    
    #Removing stopwords
    stop = stopwords.words('english')
    df[col] = df[col].apply(lambda x : ' '.join([word for word in x.split() if word not in (stop)]))

preprocessing('reviewText')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/shrikantkale/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
def product_ratings(x):
    if x['overall'] == 5 or x['overall'] == 4:
        x['ratings'] = 'Positive'
    elif x['overall'] == 3:
        x['ratings'] = 'Neutral'
    elif x['overall'] == 2 or x['overall'] == 1:
        x['ratings'] = 'Negative'
    return x

df = df.apply(product_ratings, axis = 1)
df['ratings'].value_counts()

Positive    9022
Neutral      772
Negative     467
Name: ratings, dtype: int64

In [5]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/shrikantkale/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
df['scores'] = df['reviewText'].apply(lambda reviewText: sid.polarity_scores(reviewText))
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,ratings,scores
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]",much write doe exactly supposed filter pop sou...,5,good,1393545600,"02 28, 2014",Positive,"{'neg': 0.088, 'neu': 0.68, 'pos': 0.231, 'com..."
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",product doe exactly quite affordablei realized...,5,Jake,1363392000,"03 16, 2013",Positive,"{'neg': 0.0, 'neu': 0.751, 'pos': 0.249, 'comp..."
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",primary job device block breath would otherwis...,5,It Does The Job Well,1377648000,"08 28, 2013",Positive,"{'neg': 0.12, 'neu': 0.83, 'pos': 0.05, 'compo..."
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",nice windscreen protects mxl mic prevents pop ...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014",Positive,"{'neg': 0.087, 'neu': 0.595, 'pos': 0.317, 'co..."
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",pop filter great look performs like studio fil...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014",Positive,"{'neg': 0.0, 'neu': 0.68, 'pos': 0.32, 'compou..."


In [7]:
df['compound'] = df['scores'].apply(lambda score_dict: score_dict['compound'])
df['sentiment_type']=''
df.loc[df.compound>0,'sentiment_type']='Positive'
df.loc[df.compound==0,'sentiment_type']='Neutral'
df.loc[df.compound<0,'sentiment_type']='Negative'
df

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,ratings,scores,compound,sentiment_type
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]",much write doe exactly supposed filter pop sou...,5,good,1393545600,"02 28, 2014",Positive,"{'neg': 0.088, 'neu': 0.68, 'pos': 0.231, 'com...",0.4939,Positive
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",product doe exactly quite affordablei realized...,5,Jake,1363392000,"03 16, 2013",Positive,"{'neg': 0.0, 'neu': 0.751, 'pos': 0.249, 'comp...",0.9359,Positive
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",primary job device block breath would otherwis...,5,It Does The Job Well,1377648000,"08 28, 2013",Positive,"{'neg': 0.12, 'neu': 0.83, 'pos': 0.05, 'compo...",-0.5267,Negative
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",nice windscreen protects mxl mic prevents pop ...,5,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014",Positive,"{'neg': 0.087, 'neu': 0.595, 'pos': 0.317, 'co...",0.5859,Positive
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",pop filter great look performs like studio fil...,5,No more pops when I record my vocals.,1392940800,"02 21, 2014",Positive,"{'neg': 0.0, 'neu': 0.68, 'pos': 0.32, 'compou...",0.7650,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10256,A14B2YH83ZXMPP,B00JBIVXGC,Lonnie M. Adams,"[0, 0]",great expected thank,5,Five Stars,1405814400,"07 20, 2014",Positive,"{'neg': 0.0, 'neu': 0.132, 'pos': 0.868, 'comp...",0.7650,Positive
10257,A1RPTVW5VEOSI,B00JBIVXGC,Michael J. Edelman,"[0, 0]",ive thinking trying nanoweb string wa bit put ...,5,"Long life, and for some players, a good econom...",1404259200,"07 2, 2014",Positive,"{'neg': 0.099, 'neu': 0.735, 'pos': 0.166, 'co...",0.9706,Positive
10258,AWCJ12KBO5VII,B00JBIVXGC,Michael L. Knapp,"[0, 0]",tried coated string past including elixir neve...,4,Good for coated.,1405987200,"07 22, 2014",Positive,"{'neg': 0.292, 'neu': 0.676, 'pos': 0.032, 'co...",-0.9542,Negative
10259,A2Z7S8B5U4PAKJ,B00JBIVXGC,"Rick Langdon ""Scriptor""","[0, 0]",well made elixir developed taylor guitar strin...,4,Taylor Made,1404172800,"07 1, 2014",Positive,"{'neg': 0.065, 'neu': 0.741, 'pos': 0.194, 'co...",0.9648,Positive


In [13]:
#calculating accuracy score
from sklearn.metrics import accuracy_score
print("Accuracy: ",accuracy_score(df["ratings"],df["sentiment_type"]))

#calculating precision score
from sklearn.metrics import precision_score
print("Precision score:", precision_score(df["ratings"],df["sentiment_type"], average='macro'))

#calculating precision score
from sklearn.metrics import recall_score
print("Recall score:", recall_score(df["ratings"],df["sentiment_type"], average='micro'))

#calculating f1 score   
from sklearn.metrics import f1_score
print("F1 score: ",f1_score(df["ratings"],df["sentiment_type"], average='macro')) 

Accuracy:  0.8315953610759185
Precision score: 0.4026796003027095
Recall score: 0.8315953610759185
F1 score:  0.40175394746285736
